# Stage 6

## Running the experiments

In the previous stage, our team selected `Sammon's error` function and `Genetic algorithm` and `Simulated annealing` for the experiments, so we wil use them here.

### 1. Loading and splitting the data

In [127]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [128]:
# Loading the data
df = pd.read_csv('combined-metrics.csv')

In [129]:
X = df.drop('Name', axis=1).values
y = df['Name'].values

In [130]:
print(X.shape)

(101, 38)


In [131]:
X

array([[5.42625369e+00, 3.77802360e+01, 2.97935103e+01, ...,
        1.81400000e+03, 1.58500000e+04, 0.00000000e+00],
       [2.02732240e+00, 9.52868852e+00, 6.37568306e+00, ...,
        9.22000000e+02, 2.64500000e+03, 0.00000000e+00],
       [1.67987805e+00, 1.17048780e+01, 8.17073171e+00, ...,
        2.79100000e+03, 1.14790000e+04, 0.00000000e+00],
       ...,
       [4.29198473e+00, 2.41488550e+01, 1.87232824e+01, ...,
        3.82000000e+02, 6.43600000e+03, 0.00000000e+00],
       [9.15789474e+00, 3.77017544e+01, 2.96842105e+01, ...,
        1.78000000e+02, 1.38800000e+03, 0.00000000e+00],
       [2.24528302e+00, 1.04371069e+01, 7.09119497e+00, ...,
        8.01000000e+02, 2.98200000e+03, 0.00000000e+00]])

In [132]:
y

array(['ART', 'BentoML', 'Bokeh', 'Camel', 'CatBoost', 'Causal ML',
       'Chainer', 'Computer Vision', 'D2L', 'Darts', 'Dash', 'DeepChem',
       'DeepMind Control', 'DeepPavlov', 'Detectron2', 'DIGITS',
       'DragGan', 'EasyOCR', 'ELI5', 'EvalAI', 'facenet', 'FaceSwap',
       'Fairseq', 'FastAI', 'FeatureTools', 'FiftyOne', 'gensim',
       'Giskard', 'Gluonts', 'Google Flax', 'Google JAX', 'GPT-Engineer',
       'GPTDiscord', 'Gradio', 'Gymnasium', 'Horovod', 'ImageAI',
       'imbalanced-learn', 'InsightFace', 'Kaolin', 'Kedro', 'Keras',
       'Kserve', 'Lightning', 'Ludwig', 'Mage-ai', 'Mars', 'Matplotlib',
       'metatransformer', 'Mindsdb', 'MLflow', 'Mycroft',
       'Neural Prophet', 'NNI', 'Numpy', 'ONNX', 'Open-Assistant',
       'OpenAI Baselines', 'OpenAI Python API library', 'OpenVINO',
       'Optuna', 'Paddle', 'Pandas', 'Pocker', 'Pybrain', 'PyCaret',
       'pycm', 'PyMC', 'Pyro', 'PyTensor', 'PyTorch',
       'Pytorch image models', 'qlib', 'Rasa', 'Ray', 'Reco

In [133]:
# Splitting the data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)

(80, 38)
(21, 38)


### 2. Determining the minimal subset of metrics on the training data

Sammon's error function that we will use as an objective

In [134]:
import numpy as np

# Sammon's error function
def sammons_error(X, Y):
    """
    X: Original high-dimensional data
    Y: Low-dimensional representation
    """
    # Calculate pairwise distances in X and Y
    dist_orig = np.sqrt(np.sum((X[:, np.newaxis, :] - X[np.newaxis, :, :]) ** 2, axis=-1))
    dist_lowd = np.sqrt(np.sum((Y[:, np.newaxis, :] - Y[np.newaxis, :, :]) ** 2, axis=-1))

    # Avoid division by zero
    epsilon = 1e-12

    numerator = np.sum(((dist_orig - dist_lowd) * 2) / (dist_orig + epsilon))
    denominator = np.sum(dist_orig)

    error = numerator / denominator

    return error

#### Genetic algorithm

In [135]:
def fitness_genetic(individual, X):
    # Create a subset of features based on the individual
    subset_X = X[:, np.array(individual).astype(bool)]

    # Calculate the fitness using the `sammons_error` function
    fitness = -sammons_error(X, subset_X)
    return fitness,

In [136]:
from deap import creator, base, tools, algorithms
import random
from sklearn.decomposition import PCA

# Set up the genetic algorithm
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)


toolbox = base.Toolbox()


# Functions to limit the number of features in the individual to 10
def create_individual():
    individual = [random.randint(0, 1) for _ in range(X.shape[1])]
    while sum(individual) > 10:
        indices = np.random.choice(np.arange(X.shape[1]), size=sum(individual) - 10, replace=False)
        for idx in indices:
            individual[idx] = 0
    return creator.Individual(individual)


def mate(ind1, ind2):
    child1, child2 = [toolbox.clone(ind) for ind in (ind1, ind2)]
    tools.cxTwoPoint(child1, child2)

    # Limit the number of features in the child to 10
    while sum(child1) > 10:
        indices = np.random.choice(np.arange(X.shape[1]), size=sum(child1) - 10, replace=False)
        for idx in indices:
            child1[idx] = 0
    while sum(child2) > 10:
        indices = np.random.choice(np.arange(X.shape[1]), size=sum(child2) - 10, replace=False)
        for idx in indices:
            child2[idx] = 0
    return child1, child2


def mutate(individual):
    child = toolbox.clone(individual)
    tools.mutFlipBit(child, indpb=0.05)
    while sum(child) > 10:
        indices = np.random.choice(np.arange(X.shape[1]), size=sum(child) - 10, replace=False)
        for idx in indices:
            child[idx] = 0
    return child,


toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", fitness_genetic, X=X_train)
toolbox.register("mate", mate)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selTournament, tournsize=3)

d:\Projects giperion\lsd-metrics\venv\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
d:\Projects giperion\lsd-metrics\venv\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [137]:
import numpy as np

# Run the genetic algorithm
population = toolbox.population(n=100)
hof = tools.HallOfFame(100)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

population, log = algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=100, stats=stats, halloffame=hof, verbose=True)

gen	nevals	avg         	std        	min         	max         
0  	100   	-3.12644e-06	1.47571e-06	-5.46264e-06	-4.25617e-07


1  	69    	-1.78717e-06	1.13958e-06	-5.16721e-06	-3.24598e-07
2  	56    	-9.33332e-07	6.20259e-07	-4.20687e-06	-2.73347e-07
3  	57    	-6.47685e-07	4.2254e-07 	-2.2848e-06 	-1.68969e-07
4  	64    	-5.1643e-07 	3.17264e-07	-1.97969e-06	-1.15977e-07
5  	64    	-5.01358e-07	4.9658e-07 	-3.71884e-06	-1.15977e-07
6  	69    	-4.05469e-07	3.9689e-07 	-2.01889e-06	-1.15977e-07
7  	65    	-3.45587e-07	4.84381e-07	-3.81399e-06	-9.17755e-08
8  	66    	-2.6964e-07 	4.39743e-07	-3.31674e-06	-5.2306e-08 
9  	63    	-2.2918e-07 	3.26902e-07	-1.91052e-06	-5.2306e-08 
10 	53    	-1.63985e-07	2.62122e-07	-2.01889e-06	-5.2306e-08 
11 	57    	-2.03499e-07	3.10396e-07	-1.9169e-06 	-5.2306e-08 
12 	71    	-2.51007e-07	4.97414e-07	-3.67494e-06	-5.20341e-08
13 	61    	-1.75586e-07	2.92552e-07	-1.6986e-06 	-5.2306e-08 
14 	63    	-1.91977e-07	3.53366e-07	-1.69162e-06	-4.80154e-08
15 	57    	-1.50695e-07	3.97679e-07	-3.37067e-06	-2.71188e-08
16 	51    	-1.31402e-07	2.54136e-07	-1.46413e-06	-2.71188e-08
17 	53  

In [138]:
# Get the best individual (subset of metrics)
best_individual = hof.items[0]
best_subset_genetic = [df.columns[i+1] for i, bit in enumerate(best_individual) if bit]

print("Best subset of metrics:", best_subset_genetic)
print("Best fitness:", fitness_genetic(best_individual, X_train))
print("Length of best subset:", len(best_subset_genetic))

Best subset of metrics: ['effort', 'CEE', 'LDC', 'LLDC', 'TLLOC', 'TLOC', 'TNOS', 'TCLOC', 'WarningMajor', 'WarningMinor']
Best fitness: (-1.1856353533101324e-08,)
Length of best subset: 10


#### Simulated annealing

In [139]:
import math

def fitness_annealing(X, subset_X):
    return -sammons_error(X, subset_X)

def accept_probability(curr_score, best_score, temperature):
    if curr_score > best_score:
        return 1.0
    else:
        return math.exp((curr_score - best_score) / temperature)
    
def generate_random_subset(X):
    # Generate new random subset
    subset = np.random.randint(2, size=(38,)) # 0 or 1 masks

    # Limit the number of features in the subset to 10
    while sum(subset) > 10:
        indices = np.random.choice(np.arange(X.shape[1]), size=sum(subset) - 10, replace=False)
        for idx in indices:
            subset[idx] = 0

    return subset

def annealing(X):
    # Simulated annealing parameters
    t = 10000 # Initial temperature
    t_min = 1e-3 # Minimum temperature
    cooling_factor = 0.9990 # Temperature damping factor
    max_iterations = 10000 # Maximum number of iterations
    best_subset = None
    best_score = float("-inf")
    
    for _ in range(max_iterations):

        subset = generate_random_subset(X) # Generate random subset    

        score = fitness_annealing(X, X[:,subset==1]) # Evaluate subset

        # Update best, higher is better
        if score > best_score or random.random() < accept_probability(score, best_score, t):
            best_subset = subset
            best_score = score

        # Cool temperature
        t = t * cooling_factor

        # Check if cooled enough
        if t < t_min:
            break

    return best_subset, best_score

best_subset, best_score = annealing(X_train)

print("Best subset:", best_subset)
print("Best sammon's error result:", -best_score)

Best subset: [0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0 0 1
 0]
Best sammon's error result: 1.6922541850323575e-06


In [140]:
best_subset_annealing = [df.columns[i+1] for i, bit in enumerate(best_subset) if bit]
print(best_subset_annealing)
print(len(best_subset_annealing))

['effort', 'bugs', 'CCO', 'LDC', 'CLLC', 'CEG', 'TLOC', 'TNM', 'TNPKG', 'WarningMinor']
10


In [141]:
print("Genetic algorithm result: ", best_subset_annealing)
print("Simulated annealing result:", best_subset_genetic)

X_genetic = X_train[:, np.array(best_individual).astype(bool)]
X_annealing = X_train[:, np.array(best_subset).astype(bool)]

X_genetic_test = X_test[:, np.array(best_individual).astype(bool)]
X_annealing_test = X_test[:, np.array(best_subset).astype(bool)]

print("Genetic training score:", sammons_error(X_train, X_genetic))
print("Annealing training score:", sammons_error(X_train, X_annealing))

print("Genetic validation score:", sammons_error(X_test, X_genetic_test))
print("Annealing validation score:", sammons_error(X_test, X_annealing_test))

Genetic algorithm result:  ['effort', 'bugs', 'CCO', 'LDC', 'CLLC', 'CEG', 'TLOC', 'TNM', 'TNPKG', 'WarningMinor']
Simulated annealing result: ['effort', 'CEE', 'LDC', 'LLDC', 'TLLOC', 'TLOC', 'TNOS', 'TCLOC', 'WarningMajor', 'WarningMinor']
Genetic training score: 1.1856353533101324e-08
Annealing training score: 1.6922541850323575e-06
Genetic validation score: 1.2196494464090206e-08
Annealing validation score: 1.5579669309100993e-06
